# Notebook for clustering regions & add data about birth rate and poverty

* Cluster similiar provinces within nations to larger regions to reduce computational requirements


* <font color='red'>WARNING: Clustering is arbitrary based on yearly average instead of similiarity of timeseries!</font>


In [5]:
%matplotlib inline
# Import libraries for geomatics
import geopandas as gp

# Import libraries for data handling
import pandas as pd
import numpy as np

In [6]:
pd.__version__

u'0.18.1'

In [2]:
shapefilepath =  'preprocessing_results/shapefile/'
df_shapefile_provinces = gp.GeoDataFrame.from_file(shapefilepath+'df_shapefile_provinces_monthly_norm01.shp')
df_shapefile_provinces_norm01 = df_shapefile_provinces.copy()

In [3]:
df_shapefile_provinces_norm01.columns

Index([ u'10Aegypti', u'10Albopict', u'10combi_me',  u'11Aegypti',
       u'11Albopict', u'11combined',  u'12Aegypti', u'12Albopict',
       u'12combined',   u'1Aegypti',  u'1Albopict',  u'1combined',
         u'2Aegypti',  u'2Albopict',  u'2combined',   u'3Aegypti',
        u'3Albopict',  u'3combined',   u'4Aegypti',  u'4Albopict',
        u'4combined',   u'5Aegypti',  u'5Albopict',  u'5combined',
         u'6Aegypti',  u'6Albopict',  u'6combined',   u'7Aegypti',
        u'7Albopict',  u'7combined',   u'8Aegypti',  u'8Albopict',
        u'8combined',   u'9Aegypti',  u'9Albopict',  u'9combined',
       u'Messinamea',    u'adm0_a3',    u'country',   u'geometry',
       u'perc_pover', u'population'],
      dtype='object')

In [4]:
len(df_shapefile_provinces)

4458

#### Compute yearly average for  both mosquitoes  

In [5]:
import re
def subset_df(str_filter):
    col_Aegypti = filter(lambda k: (str_filter in k), df_shapefile_provinces_norm01.columns.tolist())
    df_interactive_Aegypti = df_shapefile_provinces_norm01[col_Aegypti].copy()
    lst = [ int(re.findall('\d+', s)[0]) for s in df_interactive_Aegypti.columns ]
    rename_dict = dict(zip(df_interactive_Aegypti.columns, lst))
    df_interactive_Aegypti.rename(columns=rename_dict, inplace=True)
    return df_interactive_Aegypti

In [6]:
df_shapefile_provinces_norm01['year_all'] = subset_df('combi').mean(axis=1)

In [7]:
df_shapefile_provinces_norm01.head(1)

,10Aegypti,10Albopict,10combi_me,11Aegypti,11Albopict,11combined,12Aegypti,12Albopict,12combined,1Aegypti,...,9Aegypti,9Albopict,9combined,Messinamea,adm0_a3,country,geometry,perc_pover,population,year_all
0,0.308523,0.041353,0.308523,0.305445,0.100243,0.305445,0.231323,0.110166,0.231323,0.109213,...,0.122034,0.097954,0.1446,0.226241,ABW,Aruba,POLYGON ((-69.99693762899992 12.57758209800004...,12.899919,87896.878144,0.1577


#### Discretize elements into equal-ranged buckets based on Env_suitability. 

In [8]:
pd.cut(df_shapefile_provinces_norm01['year_all'], 10).value_counts()

(-8.27e-05, 0.0244]    1357
(0.121, 0.145]          606
(0.0971, 0.121]         482
(0.145, 0.17]           471
(0.0244, 0.0486]        431
(0.0486, 0.0728]        408
(0.0728, 0.0971]        375
(0.17, 0.194]           241
(0.194, 0.218]           69
(0.218, 0.242]           18
Name: year_all, dtype: int64

In [9]:
df_shapefile_provinces_norm01['category'] = pd.cut(df_shapefile_provinces_norm01['year_all'], 10)

#### Group first two groups together 

In [10]:
set(df_shapefile_provinces_norm01['category'].tolist())
m = {'(-8.27e-05, 0.0244]': 1,  
     '(0.0244, 0.0486]': 1, 
      '(0.0486, 0.0728]': 2, '(0.0728, 0.0971]': 3,  '(0.0971, 0.121]' :4 , 
    '(0.121, 0.145]' : 5, '(0.145, 0.17]': 6, '(0.17, 0.194]': 7, '(0.194, 0.218]': 8 , '(0.218, 0.242]':  9}

dic = lambda x: m[x]
df_shapefile_provinces_norm01['category'] = df_shapefile_provinces_norm01['category'].apply(dic)

#### Group all elements by the country and its Env_suitability bin (>10 min computation period)
Compute the cumulative population, mean poverty and mean Env_suitability of each group and add as extra column to each element of df

In [11]:
col_Zika = filter(lambda k: ('Aegypti' in k) or ('Albopi' in k) or  ('combi' in k) or ('Messina' in k), df_shapefile_provinces_norm01.columns.tolist())

aggregation = dict(zip(col_Zika[::1], [np.mean]*len(col_Zika)))
aggregation['population'] = np.sum
aggregation['perc_pover'] = np.mean

In [12]:
df_shapefile_provinces_norm01 = df_shapefile_provinces_norm01.dissolve(by=['country', 'category'],
                                           aggfunc=aggregation,
                                           as_index=True)
df_shapefile_provinces_norm01.reset_index(inplace=True)


In [13]:
len(df_shapefile_provinces_norm01)

765

#### Add continent and country-code to shapefile with spatial join

- There may be easier ways to achieve this than reading and mapping this from the country shapefile from naturalearthdata but it works.


In [14]:
shapefile = 'raw_data/shapefile/naturalearthdata/ne_50m_admin_0_countries.shp'
df_shapefile_countries = gp.GeoDataFrame.from_file(shapefile)
cols = ['geometry', 'continent', 'adm0_a3', 'admin']
df_shapefile_countries = df_shapefile_countries[cols]

df_shapefile_provinces_norm01.crs = df_shapefile_countries.crs

dic_ccode_continent = dict(zip(df_shapefile_countries['adm0_a3'],df_shapefile_countries['continent']))
dic_country_ccode = dict(zip(df_shapefile_countries.admin,df_shapefile_countries['adm0_a3']))

df_shapefile_provinces_norm01["adm0_a3"] = df_shapefile_provinces_norm01["country"].map(dic_country_ccode)
df_shapefile_provinces_norm01["continent"] = df_shapefile_provinces_norm01["adm0_a3"].map(dic_ccode_continent)

In [21]:
print 'CRS', df_shapefile_provinces_norm01.crs
print 'Number of map elements: ', len(df_shapefile_provinces_norm01  )
df_shapefile_provinces_norm01.head(1)

CRS {'init': u'epsg:4326'}
Number of map elements:  765


,country,category,geometry,5Aegypti,population,12combined,12Aegypti,2combined,8Aegypti,9combined,...,3combined,10combi_me,4Albopict,3Aegypti,11Albopict,8Albopict,7combined,11combined,adm0_a3,continent
0,Afghanistan,1,"POLYGON ((69.48586754674983 34.15900055643047,...",0.013484,2.715357e+07,0.007285,0.003349,0.005081,0.022927,0.022721,...,0.005834,0.036989,0.001669,0.005626,0.01632,0.01097,0.025059,0.017434,AFG,Asia


#### Mapping did not work for all provinces, hence derive from spatial join. Note spatial join does work relatively poorly
<font color='red'>WARNING: Dirty Hack sjoin based on centroid of province!</font>


In [22]:
len(df_shapefile_provinces_norm01[df_shapefile_provinces_norm01.continent.isnull()])

14

NameError: name 'pd' is not defined

In [23]:
df_nan = df_shapefile_provinces_norm01[df_shapefile_provinces_norm01.continent.isnull()]

In [25]:
df_nan.drop(['continent', 'adm0_a3'], axis=1, inplace=True)
df_shapefile_countries.drop('admin', axis=1, inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
df_nan['centroid'] = df_nan.geometry.centroid
df_nan['shape'] = df_nan['geometry']
df_nan['geometry']= df_nan['centroid'] 
df_nan = gp.sjoin(df_nan, df_shapefile_countries, how='left')
df_nan['geometry'] = df_nan['shape']

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [27]:
df_nan.drop(['index_right', 'centroid', 'shape'], axis=1, inplace=True )

In [28]:
df_shapefile_provinces_norm01 = pd.concat([df_shapefile_provinces_norm01.dropna(), df_nan.dropna()])

#### Add data on birth rate to estimate Microcephaly cases

In [29]:
df_birth_rate_crude = pd.read_csv('raw_data/Worldbank_dataset/BirthRateCrude_WorldBank.csv', 
                                  skiprows=4,
                                  header=0,
                                 usecols=["Country Code", '2014'])
df_birth_rate_crude.rename(
    columns={'Country Code': 'adm0_a3', '2014' :'Birth rate, crude (per thousand people)'}, inplace=True)

# keep everything from the left frame, pulling in the value from the right frame where the keys match up. 
df_shapefile_provinces_norm01 = df_shapefile_provinces_norm01.merge(df_birth_rate_crude,
                                                                    on='adm0_a3',
                                                                    how='left')

# replace not find key with average birth rate
df_shapefile_provinces_norm01['Birth rate, crude (per thousand people)'].fillna(
    (df_shapefile_provinces_norm01['Birth rate, crude (per thousand people)'].mean()), inplace=True)

df_shapefile_provinces_norm01['pregnancies'] = (df_shapefile_provinces_norm01['Birth rate, crude (per thousand people)'] 
                                                * (1.0/1000) * (9.0/12) 
                                                * df_shapefile_provinces_norm01['population'])

#### Add data on poverty  (defined as people with lessa than 3.1 Dollar per day) according to World Bank Development indicators (country level)

In [30]:
df_birth_rate_crude = pd.read_csv('raw_data/Worldbank_dataset/Poverty_3.10Dollar_a_day.csv', 
                                  skiprows=4,
                                  header=0,
                                 usecols=["Country Code", '2013'])
df_birth_rate_crude.rename(columns={'Country Code': 'adm0_a3', '2013' :'Poverty_WB'}, inplace=True)

# keep everything from the left frame, pulling in the value from the right frame where the keys match up. 
df_shapefile_provinces_norm01 = df_shapefile_provinces_norm01.merge(df_birth_rate_crude,
                                                                    on='adm0_a3',
                                                                    how='left') ## DO NOT USE PD.MERGE with gp !!!!!

# replace not find key with average birth rate
df_shapefile_provinces_norm01['Poverty_WB'].fillna(
    (df_shapefile_provinces_norm01['Poverty_WB'].mean()), inplace=True)

In [31]:
print 'Number of map elements: ', len(df_shapefile_provinces_norm01  )
df_shapefile_provinces_norm01.head(1)

Number of map elements:  763


,10Aegypti,10Albopict,10combi_me,11Aegypti,11Albopict,11combined,12Aegypti,12Albopict,12combined,1Aegypti,...,adm0_a3,category,continent,country,geometry,perc_pover,population,"Birth rate, crude (per thousand people)",pregnancies,Poverty_WB
0,0.014458,0.032256,0.036989,0.003544,0.01632,0.017434,0.003349,0.005992,0.007285,0.003574,...,AFG,1,Asia,Afghanistan,"POLYGON ((69.48586754674983 34.15900055643047,...",74.586433,2.715357e+07,34.225,696998.21022,10.134875


In [32]:
np.unique(df_shapefile_provinces_norm01.continent)

array([u'Africa', u'Asia', u'Europe', u'North America', u'Oceania',
       u'Seven seas (open ocean)', u'South America'], dtype=object)

#### Clean geodaframe from no longer needed columns

In [34]:
df_shapefile_provinces_norm01.drop(['category'], axis=1, inplace=True )

#### Draw polygons more efficiently with matplotlib, create dictionary with index as key and matplotlib polygon as value

In [35]:
from descartes import PolygonPatch
from shapely.geometry import MultiPolygon

df_shapefile_provinces_norm01 ["mpl_polygon"] = np.nan
df_shapefile_provinces_norm01 ['mpl_polygon'] = df_shapefile_provinces_norm01 ['mpl_polygon'].astype(object)
for self_index, self_row_df in df_shapefile_provinces_norm01 .iterrows():
    m_polygon = self_row_df['geometry']
    poly=[]
    if m_polygon.geom_type == 'MultiPolygon':
        for pol in m_polygon:
            poly.append(PolygonPatch(pol))
    else:
        poly.append(PolygonPatch(m_polygon))
    df_shapefile_provinces_norm01.set_value(self_index, 'mpl_polygon', poly)

In [36]:
import re
def subset_df(str_filter):
    col_Aegypti = filter(lambda k: (str_filter in k), df_shapefile_provinces_norm01.columns.tolist())
    df_interactive_Aegypti = df_shapefile_provinces_norm01[col_Aegypti].copy()
    lst = [ int(re.findall('\d+', s)[0]) for s in df_interactive_Aegypti.columns ]
    rename_dict = dict(zip(df_interactive_Aegypti.columns, lst))
    df_interactive_Aegypti.rename(columns=rename_dict, inplace=True)
    df_interactive_Aegypti = df_interactive_Aegypti.append(
                                        pd.DataFrame(1,
                                                     index=np.arange(1),
                                                     columns=df_interactive_Aegypti.columns))
    return df_interactive_Aegypti

In [37]:
frames = [subset_df('Aegypti'), subset_df('Albopict'), subset_df( 'combi')]
multiindex_plot_suitability = pd.concat(frames, keys=['Aegypti', 'Albopictus', 'Both'])
multiindex_plot_suitability.sort_index(axis=1, inplace=True)

In [38]:
multiindex_plot_suitability.to_csv('preprocessing_results/csv_files/Env_suitability_monthly.csv')

In [39]:
df_shapefile_provinces_norm01.rename(columns={'adm0_a3': 'country_code'}, inplace=True)
col_contain_digit= df_shapefile_provinces_norm01.columns.to_series().apply(lambda s: 
                                                                           any(i.isdigit() for i in s))
col_contain_digit = ~col_contain_digit
df_shapefile_provinces_norm01 = df_shapefile_provinces_norm01.loc[:, col_contain_digit.tolist()]

In [40]:
df_shapefile_provinces_norm01.to_pickle('preprocessing_results/pickle_obj/gdf_map_elements.pkl')

In [44]:
df_shapefile_provinces_norm01.drop(['mpl_polygon'], axis=1, inplace=True)

In [46]:
df_shapefile_provinces_norm01.to_file(shapefilepath +
                                      'df_shapefile_provinces_monthly_norm01_custered_WB.shp',
                                      driver="ESRI Shapefile")